# Background Operations query with Expression to delete a bin

# Import to run %sh magic cell

In [1]:
import io.github.spencerpark.ijava.IJava;
import io.github.spencerpark.jupyter.kernel.magic.common.Shell;
IJava.getKernelInstance().getMagics().registerMagics(Shell.class);

# Add Java Client POM Dependency

In [2]:
%%loadFromPOM
<dependencies>
  <dependency>
    <groupId>com.aerospike</groupId>
    <artifactId>aerospike-client-jdk8</artifactId>
    <version>8.1.1</version>
  </dependency>
</dependencies>

# Add required Imports

In [3]:
import com.aerospike.client.AerospikeClient;
import com.aerospike.client.policy.WritePolicy;
import com.aerospike.client.Bin;
import com.aerospike.client.Key;
import com.aerospike.client.Record;
import com.aerospike.client.Value;
System.out.println("Client modules imported.");

Client modules imported.


# Generate Test Data

In [4]:
%sh asadm --enable -e "manage truncate ns test --no-warn" -h "127.0.0.1"

In [5]:
%sh aql -c "set key_send true"

In [10]:
%sh aql -f "../aqlScripts/insert.aql"

In [7]:
AerospikeClient client = new AerospikeClient("localhost", 3000);
System.out.println("Initialized the client and connected to the cluster.");

Key key = new Key("test", "testset", "key1");
System.out.println("Working with record key:");
System.out.println(key);

Record record = client.get(null, key);
System.out.println("Read back the record.");

System.out.println("Record values are:");
System.out.println(record);

Initialized the client and connected to the cluster.
Working with record key:
test:testset:key1:bf6c1d13e7cd10c5bd022d27e7df170c0bccd6e1
Read back the record.
Record values are:
(gen:1),(exp:450856024),(bins:(name:Jack),(age:26))


In [8]:
%sh asadm --enable -e "manage sindex create numeric idx_age ns test set testset bin age"

In [11]:
//Needed imports
import com.aerospike.client.query.Statement;
import com.aerospike.client.query.Filter;
import com.aerospike.client.Operation;
import com.aerospike.client.Bin;
import com.aerospike.client.task.ExecuteTask;

//Run SI query
Statement stmt = new Statement();
stmt.setNamespace("test");
stmt.setSetName("testset");
stmt.setFilter(Filter.range("age", 20,60));
Operation[] ops = new Operation[1];
Bin ageBinNull = Bin.asNull("age");
ops[0] = Operation.put(ageBinNull);
stmt.setOperations(ops);
WritePolicy wp = new WritePolicy();

ExecuteTask task = client.execute(wp, stmt);


# Check
In terminal, run aql. 

aql> select * from test.testset 

Records who age bin is in the Filter range gets deleted.

# Cleanup

In [12]:
%sh asadm --enable -e "manage sindex delete idx_age ns test set testset"

In [13]:
%sh asadm --enable -e "manage truncate ns test --no-warn" -h "127.0.0.1"